In [82]:
import numpy as np
import pickle
import pandas as pd
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk
from sklearn.naive_bayes import GaussianNB

In [2]:
def read_kImage(path):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(path)
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    return ct_scan

In [3]:
def generate_training_tables():
    basedir = "../1/train/ProstateX-TrainingLesionInformationv2/ProstateX-TrainingLesionInformationv2"
    findingsfile = "{}/{}".format(basedir,"ProstateX-Findings-Train.csv")
    imagesfile = "{}/{}".format(basedir,"ProstateX-Images-Train.csv")
    kimagesfile = "{}/{}".format(basedir,"ProstateX-Images-KTrans-Train.csv")
    #Generamos las 3 tablas básicas a partir de los archivos.cvs
    findings_table = pd.read_csv(findingsfile)
    images_table = pd.read_csv(imagesfile)
    kimages_table = pd.read_csv(kimagesfile)
    #Operación de inner join entre tablas.
    join_kimages_table = pd.merge(left=findings_table, right= kimages_table, how ="inner", on=["ProxID","fid","pos"])
    join_images_table =  pd.merge(left=findings_table, right=  images_table, how ="inner", on=["ProxID","fid","pos"])
    #agregamos una columna para insertar la imagen
    join_images_table["MRI"] = pd.Series(np.zeros((len(join_images_table),2,2,2)).tolist(), index= join_images_table.index)
    join_images_table["State"] = pd.Series(False, index= join_images_table.index)
    join_kimages_table["Kimg"] = pd.Series(np.zeros((len(join_kimages_table),2,2,2)).tolist(), index= join_kimages_table.index)
    join_kimages_table["State"] = pd.Series(False, index= join_kimages_table.index)
    return join_images_table, join_kimages_table

In [4]:
def extract_voxel_data(series_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames( series_path )
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image_3d = sitk.GetArrayFromImage(image)
    return image_3d

In [5]:
def extract_slice_data(img_3d, ijk):
    state = False
    k_string = ijk.split()[2]
    max_bandwith = img_3d.shape[0]
    possible_k = int (k_string)
    if (possible_k >= max_bandwith):
        possible_k = max_bandwith -1
        slice_index = possible_k
        state = True
        print("The ijk requested is {}, there was an error since the shapes of the image is {}".format(ijk,str(img_3d.shape)))
    else:
        slice_index = possible_k
        print("The ijk requested is {},OK since shapes of the image is {}".format(ijk,str(img_3d.shape)))
    return state, img_3d[slice_index,:,:]

In [6]:
def set_MRI_value(index,image,state):
    join_images_table.at[index,"MRI"] = image
    join_images_table.at[index,"State"] = state

In [7]:
def set_kImg_value(index,image,state):
    join_kimages_table.at[index,"Kimg"] = image
    join_kimages_table.at[index,"State"] = state

In [8]:
def insert_k_images():
    for row in range(len(join_kimages_table)):
        path = "../1/Train/ProstateXKtrains-train-fixed"
        patient_id = join_kimages_table.ProxID.iloc[row]
        ijk = join_kimages_table.ijk.iloc[row]
        path = "{}/{}".format(path,patient_id)
        path = "{}/{}-Ktrans.mhd".format(path,patient_id)
        kimage_3d = read_kImage(path)
        state, kimage_2d = extract_slice_data(kimage_3d,ijk)
        set_kImg_value(row,kimage_2d,state)
        print("Se ha cargado la Kimagen2d #{} para el paciente {} ".format(row,patient_id))
        del path,patient_id,kimage_3d,state


In [9]:
def insert_MRI_images():
    for row in range (len(join_images_table)):
        patient_id = join_images_table.ProxID.iloc[row]
        series_name = join_images_table.DCMSerDescr.iloc[row]
        series_id = join_images_table.DCMSerNum.iloc[row]
        slice_ijk = join_images_table.ijk.iloc[row]
        path = "../1/Train/PROSTATEx/{}/".format(patient_id)#Selects the Patient ID
        path = "{}/{}".format(path,os.listdir(path)[0])#Selects the default folder inside the patient
        sequences = os.listdir(path) # Selects all the sequences
        sequence = [ sequence for sequence in sequences if sequence[0:len(str(series_id))] == str(series_id) ] [0]
        # Selects the only sequence that matches the condition.
        path = "{}/{}".format(path,sequence)
        image_3d = extract_voxel_data(path)
        state, image_2d = extract_slice_data(image_3d,slice_ijk)
        set_MRI_value(row,image_2d,state)
        print("Se ha cargado la imagen2d #{} para el paciente {} serie: {} ".format(row,patient_id,series_name))
        del path, patient_id, series_name, series_id,sequence,sequences, image_3d, image_2d

In [209]:
def get_training_testing_tables():
    number_of_cancer_samples=len(join_kimages_table[(join_kimages_table["ClinSig"] == True) & (join_kimages_table["zone"]=="PZ")])
    number_of_non_cancer_samples = len(join_kimages_table[(join_kimages_table["ClinSig"] == False) & (join_kimages_table["zone"]=="PZ")])
    print("We have {} cancer samples and {} non-cancer samples".format(number_of_cancer_samples, number_of_non_cancer_samples))
    training_cancer_samples = number_of_cancer_samples//2
    training_non_cancer_samples = number_of_non_cancer_samples//2
    training_cancer_samples_index = np.random.randint(0,number_of_cancer_samples-1,training_cancer_samples)
    training_non_cancer_samples_index = np.random.randint(0,number_of_non_cancer_samples-1,training_non_cancer_samples)
    training_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == True) & (join_kimages_table["zone"]=="PZ")].iloc[training_cancer_samples_index]
    training_non_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == False) & (join_kimages_table["zone"]=="PZ")].iloc[training_non_cancer_samples_index]
    #we get the posible samples that are not taken on the training set.
    samples = [i for i in range(training_cancer_samples)]
    test_cancer_samples_index = [sample for sample in samples if sample not in training_cancer_samples_index]
    test_cancer_samples_index = np.array(test_cancer_samples_index)
    samples = [i for i in range(training_non_cancer_samples)]
    test_non_cancer_samples_index = [sample for sample in samples if sample not in training_non_cancer_samples_index]
    test_non_cancer_samples_index = np.array(test_non_cancer_samples_index)
    test_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == True) & (join_kimages_table["zone"]=="PZ")].iloc[test_cancer_samples_index]
    test_non_cancer_samples_table = join_kimages_table[(join_kimages_table["ClinSig"] == False) & (join_kimages_table["zone"]=="PZ")].iloc[test_non_cancer_samples_index]
    del samples,test_cancer_samples_index,test_non_cancer_samples_index, training_cancer_samples_index
    training_table = training_cancer_samples_table.append(training_non_cancer_samples_table)
    testing_table = test_cancer_samples_table.append(test_non_cancer_samples_table)
    training_table = training_table[training_table["State"] == False]
    testing_table = testing_table[testing_table["State"]== False]
    training_table = training_table.sample(frac=1).reset_index(drop=True)
    testing_table = testing_table.sample(frac=1).reset_index(drop=True)

    return training_table,testing_table

In [11]:
join_images_table, join_kimages_table =  generate_training_tables()


In [12]:
insert_k_images()

The ijk requested is 56 75 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #0 para el paciente ProstateX-0000 
The ijk requested is 51 61 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #1 para el paciente ProstateX-0001 
The ijk requested is 52 73 11,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #2 para el paciente ProstateX-0002 
The ijk requested is 68 72 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #3 para el paciente ProstateX-0002 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #4 para el paciente ProstateX-0003 
The ijk requested is 51 54 13,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #5 para el paciente ProstateX-0003 
The ijk requested is 74 56 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #6 para el paciente ProstateX-0004 
The ijk requested is 69 

The ijk requested is 46 75 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #63 para el paciente ProstateX-0039 
The ijk requested is 50 63 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #64 para el paciente ProstateX-0040 
The ijk requested is 68 76 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #65 para el paciente ProstateX-0040 
The ijk requested is 74 68 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #66 para el paciente ProstateX-0040 
The ijk requested is 50 79 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #67 para el paciente ProstateX-0041 
The ijk requested is 70 58 4,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #68 para el paciente ProstateX-0042 
The ijk requested is 55 75 13,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #69 para el paciente ProstateX-0043 
The ijk requested

The ijk requested is 68 61 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #133 para el paciente ProstateX-0090 
The ijk requested is 58 66 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #134 para el paciente ProstateX-0091 
The ijk requested is 53 60 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #135 para el paciente ProstateX-0092 
The ijk requested is 67 77 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #136 para el paciente ProstateX-0093 
The ijk requested is 65 76 12,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #137 para el paciente ProstateX-0093 
The ijk requested is 52 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #138 para el paciente ProstateX-0093 
The ijk requested is 64 52 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #139 para el paciente ProstateX-0094 
The ijk req

The ijk requested is 53 73 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #193 para el paciente ProstateX-0128 
The ijk requested is 56 67 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #194 para el paciente ProstateX-0128 
The ijk requested is 68 68 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #195 para el paciente ProstateX-0128 
The ijk requested is 78 72 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #196 para el paciente ProstateX-0129 
The ijk requested is 41 77 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #197 para el paciente ProstateX-0130 
The ijk requested is 77 78 13,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #198 para el paciente ProstateX-0130 
The ijk requested is 74 72 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #199 para el paciente ProstateX-0130 
The ijk requ

The ijk requested is 57 88 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #259 para el paciente ProstateX-0161 
The ijk requested is 74 70 5,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #260 para el paciente ProstateX-0161 
The ijk requested is 53 69 8,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #261 para el paciente ProstateX-0161 
The ijk requested is 60 74 3,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #262 para el paciente ProstateX-0162 
The ijk requested is 73 54 6,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #263 para el paciente ProstateX-0163 
The ijk requested is 63 84 7,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #264 para el paciente ProstateX-0163 
The ijk requested is 68 84 9,OK since shapes of the image is (16, 128, 128)
Se ha cargado la Kimagen2d #265 para el paciente ProstateX-0163 
The ijk reque

The ijk requested is 52 64 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la Kimagen2d #329 para el paciente ProstateX-0200 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la Kimagen2d #330 para el paciente ProstateX-0201 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la Kimagen2d #331 para el paciente ProstateX-0202 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la Kimagen2d #332 para el paciente ProstateX-0202 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la Kimagen2d #333 para el paciente ProstateX-0203 
The ijk requested is 71 65 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la Kimagen2d #334 para el paciente ProstateX-0203 


In [13]:
insert_MRI_images()

The ijk requested is 36 72 9,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #0 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #1 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #2 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #3 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 36 72 9,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #4 para el paciente ProstateX-0000 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 139 166 12,OK since shapes of the image is (15, 320, 320)
Se ha cargado la imagen2d #5 para el paciente ProstateX-0000 serie: t2_tse_cor

The ijk requested is 149 130 0,OK since shapes of the image is (13, 256, 256)
Se ha cargado la imagen2d #49 para el paciente ProstateX-0003 serie: t2_localizer 
The ijk requested is 199 154 14,OK since shapes of the image is (17, 320, 320)
Se ha cargado la imagen2d #50 para el paciente ProstateX-0003 serie: t2_tse_cor 
The ijk requested is 168 165 3,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #51 para el paciente ProstateX-0003 serie: t2_tse_sag 
The ijk requested is 240 205 12,OK since shapes of the image is (21, 384, 384)
Se ha cargado la imagen2d #52 para el paciente ProstateX-0003 serie: t2_tse_tra 
The ijk requested is 80 69 10,OK since shapes of the image is (16, 128, 128)
Se ha cargado la imagen2d #53 para el paciente ProstateX-0003 serie: tfl_3d PD ref_tra_1.5x1.5_t3 
The ijk requested is 33 57 16,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #54 para el paciente ProstateX-0003 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requ

The ijk requested is 50 69 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #100 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST_ADC 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #101 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #102 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (57, 128, 84)
Se ha cargado la imagen2d #103 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDIST 
The ijk requested is 50 69 11,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #104 para el paciente ProstateX-0006 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 144 146 0,OK since shapes of the image is (3, 256, 256)
Se ha cargado la imagen2d #105 para el paciente ProstateX-0006 se

The ijk requested is 44 64 13,OK since shapes of the image is (19, 128, 84)
Se ha cargado la imagen2d #150 para el paciente ProstateX-0009 serie: ep2d_diff_tra_DYNDISTCALC_BVAL 
The ijk requested is 147 162 1,OK since shapes of the image is (3, 256, 256)
Se ha cargado la imagen2d #151 para el paciente ProstateX-0009 serie: t2_loc sag 
The ijk requested is 149 152 0,OK since shapes of the image is (13, 256, 256)
Se ha cargado la imagen2d #152 para el paciente ProstateX-0009 serie: t2_localizer 
The ijk requested is 170 151 12,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #153 para el paciente ProstateX-0009 serie: t2_tse_cor 
The ijk requested is 143 155 9,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #154 para el paciente ProstateX-0009 serie: t2_tse_sag 
The ijk requested is 197 191 14,OK since shapes of the image is (21, 384, 384)
Se ha cargado la imagen2d #155 para el paciente ProstateX-0009 serie: t2_tse_tra 
The ijk requested i

The ijk requested is 25 67 11,OK since shapes of the image is (21, 128, 84)
Se ha cargado la imagen2d #203 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX_ADC 
The ijk requested is 25 67 11,OK since shapes of the image is (63, 128, 84)
Se ha cargado la imagen2d #204 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX 
The ijk requested is 25 67 11,OK since shapes of the image is (63, 128, 84)
Se ha cargado la imagen2d #205 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX 
The ijk requested is 25 67 11,OK since shapes of the image is (63, 128, 84)
Se ha cargado la imagen2d #206 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIX 
The ijk requested is 25 67 11,OK since shapes of the image is (21, 128, 84)
Se ha cargado la imagen2d #207 para el paciente ProstateX-0013 serie: ep2d_diff_tra_DYNDIST_MIXCALC_BVAL 
The ijk requested is 137 126 5, there was an error since the shapes of the image is (3, 256, 256)
Se ha cargado la imagen2

...

The ijk requested is 53 53 10,OK since shapes of the image is (60, 106, 128)
Se ha cargado la imagen2d #3602 para el paciente ProstateX-0200 serie: diffusie-3Scan-4bval_fs 
The ijk requested is 53 53 10,OK since shapes of the image is (20, 106, 128)
Se ha cargado la imagen2d #3603 para el paciente ProstateX-0200 serie: diffusie-3Scan-4bval_fsCALC_BVAL 
The ijk requested is 257 245 1,OK since shapes of the image is (13, 512, 512)
Se ha cargado la imagen2d #3604 para el paciente ProstateX-0200 serie: t2_localizer_prostate 
The ijk requested is 122 145 7,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #3605 para el paciente ProstateX-0200 serie: t2_tse_cor 
The ijk requested is 152 151 17,OK since shapes of the image is (21, 320, 320)
Se ha cargado la imagen2d #3606 para el paciente ProstateX-0200 serie: t2_tse_sag 
The ijk requested is 120 159 11,OK since shapes of the image is (21, 320, 320)
Se ha cargado la imagen2d #3607 para el paciente ProstateX-0200 serie: 

The ijk requested is 257 264 0,OK since shapes of the image is (13, 512, 512)
Se ha cargado la imagen2d #3651 para el paciente ProstateX-0201 serie: t2_localizer_prostate 
The ijk requested is 194 135 8,OK since shapes of the image is (17, 320, 320)
Se ha cargado la imagen2d #3652 para el paciente ProstateX-0201 serie: t2_tse_cor 
The ijk requested is 164 148 3,OK since shapes of the image is (19, 320, 320)
Se ha cargado la imagen2d #3653 para el paciente ProstateX-0201 serie: t2_tse_sag 
The ijk requested is 191 163 16,OK since shapes of the image is (23, 320, 320)
Se ha cargado la imagen2d #3654 para el paciente ProstateX-0201 serie: t2_tse_tra 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3655 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
The ijk requested is 74 65 10,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3656 para el paciente ProstateX-0201 serie: tfl_3d dynamisch fast 
Th

The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3700 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3701 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3702 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3703 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3704 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 67 61 6,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3705 para el paciente ProstateX-0202 serie: tfl

The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3761 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3762 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3763 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3764 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3765 para el paciente ProstateX-0202 serie: tfl_3d dynamisch fast 
The ijk requested is 60 74 5,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3766 para el paciente ProstateX-0202 serie: tfl

The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3811 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3812 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3813 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3814 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3815 para el paciente ProstateX-0203 serie: tfl_3d dynamisch fast 
The ijk requested is 59 56 7,OK since shapes of the image is (12, 128, 128)
Se ha cargado la imagen2d #3816 para el paciente ProstateX-0203 serie: tfl

In [14]:

MRI_problems_table = join_images_table[join_images_table["State"] == True] 
print("There were {} problems".format(len(MRI_problems_table)))
percent = 100*float(len(MRI_problems_table)/len(join_images_table))

print("It was the  {0:.5f}% of the dataset".format(percent))
display(MRI_problems_table[["ProxID","DCMSerDescr","ijk","State"]])


There were 88 problems
It was the  2.27449% of the dataset


ProxID                     DCMSerDescr         ijk  State
14    ProstateX-0001                      t2_loc sag   126 145 3   True
27    ProstateX-0002                      t2_loc sag   135 132 3   True
162   ProstateX-0010                      t2_loc sag   129 154 4   True
208   ProstateX-0013                      t2_loc sag   137 126 5   True
311   ProstateX-0022                      t2_loc sag   141 135 4   True
321   ProstateX-0023                      t2_loc sag   137 131 4   True
349   ProstateX-0025       ep2d_diff_tra_DYNDIST_ADC    61 66 25   True
357   ProstateX-0025  ep2d_diff_tra_DYNDISTCALC_BVAL    61 66 25   True
360   ProstateX-0025                      t2_tse_cor  186 163 17   True
363   ProstateX-0025                      t2_tse_tra  266 200 26   True
365   ProstateX-0025    tfl_3d PD ref_tra_1.5x1.5_t3    89 67 24   True
377   ProstateX-0025                      t2_tse_cor  185 164 17   True
378   ProstateX-0025                      t2_tse_cor  168 248 21   True
395   ProstateX-0025                      t2_tse_cor  159 161 18   True
396   ProstateX-0025                      t2_tse_cor  141 245 22   True
414   ProstateX-0025                      t2_tse_cor  114 244 18   True
480   ProstateX-0029                      t2_tse_cor  172 122 17   True
520   ProstateX-0031                      t2_loc sag   146 125 3   True
532   ProstateX-0032                      t2_tse_cor  193 128 15   True
572   ProstateX-0035                      t2_loc sag   130 147 4   True
619   ProstateX-0038                      t2_loc sag   130 133 3   True
641   ProstateX-0038                      t2_loc sag   140 138 3   True
652   ProstateX-0039                      t2_loc sag   144 119 4   True
690   ProstateX-0041                      t2_tse_cor  129 191 17   True
738   ProstateX-0046                      t2_loc sag   130 129 4   True
916   ProstateX-0061                      t2_loc sag   138 119 3   True
1007  ProstateX-0067                      t2_loc sag   143 149 4   True
1045  ProstateX-0069                      t2_loc sag   135 142 3   True
1103  ProstateX-0074                      t2_tse_cor  176 183 15   True
1134  ProstateX-0077                      t2_loc sag   154 122 4   True
...              ...                             ...         ...    ...
2419  ProstateX-0148                      t2_loc sag   150 126 4   True
2422  ProstateX-0148                      t2_tse_tra  149 206 24   True
2432  ProstateX-0149                      t2_tse_cor  194 166 15   True
2444  ProstateX-0149                      t2_tse_cor  168 166 15   True
2472  ProstateX-0151                      t2_tse_cor  124 191 19   True
2536  ProstateX-0154                      t2_tse_cor  128 170 17   True
2540  ProstateX-0154       ep2d_diff_tra_DYNDIST_ADC    37 70 19   True
2544  ProstateX-0154  ep2d_diff_tra_DYNDISTCALC_BVAL    37 70 19   True
2548  ProstateX-0154    tfl_3d PD ref_tra_1.5x1.5_t3    57 72 16   True
2554  ProstateX-0155                      t2_tse_cor  179 178 17   True
2573  ProstateX-0157                      t2_loc sag   147 127 3   True
2575  ProstateX-0157                      t2_tse_cor  143 148 15   True
2595  ProstateX-0158                      t2_loc sag   147 131 4   True
2628  ProstateX-0159                      t2_loc sag   135 134 4   True
2661  ProstateX-0161                      t2_tse_cor  147 174 21   True
2681  ProstateX-0161                      t2_loc sag   128 124 3   True
2710  ProstateX-0163                      t2_tse_cor  157 166 15   True
2719  ProstateX-0163                      t2_tse_cor  170 146 15   True
2818  ProstateX-0171                      t2_tse_cor  208 164 18   True
2848  ProstateX-0172                      t2_loc sag   136 134 4   True
2859  ProstateX-0172                      t2_loc sag   146 138 3   True
2881  ProstateX-0173                      t2_loc sag   129 122 3   True
2895  ProstateX-0173                      t2_loc sag   134 138 3   True
2998  ProstateX-0179                      t2_loc sag  

In [15]:
k_problems_table = join_kimages_table[join_kimages_table["State"] == True] 
print("There were {} problems".format(len(k_problems_table)))
percent = 100*float(len(k_problems_table)/len(join_kimages_table))

print("It was the  {0:.5f}% of the dataset".format(percent))
display(k_problems_table[["ProxID","ijk","State"]])

There were 3 problems
It was the  0.89552% of the dataset


ProxID       ijk  State
34   ProstateX-0025  89 67 24   True
159  ProstateX-0105  63 78 17   True
248  ProstateX-0154  57 72 16   True

In [208]:
patient_problems = MRI_problems_table.ProxID.unique()
print(patient_problems)
candidate_table = join_images_table
for patient in patient_problems:
    candidate_table = candidate_table[candidate_table["ProxID"] != patient]
display(candidate_table)

['ProstateX-0001' 'ProstateX-0002' 'ProstateX-0010' 'ProstateX-0013'
 'ProstateX-0022' 'ProstateX-0023' 'ProstateX-0025' 'ProstateX-0029'
 'ProstateX-0031' 'ProstateX-0032' 'ProstateX-0035' 'ProstateX-0038'
 'ProstateX-0039' 'ProstateX-0041' 'ProstateX-0046' 'ProstateX-0061'
 'ProstateX-0067' 'ProstateX-0069' 'ProstateX-0074' 'ProstateX-0077'
 'ProstateX-0082' 'ProstateX-0084' 'ProstateX-0085' 'ProstateX-0086'
 'ProstateX-0088' 'ProstateX-0092' 'ProstateX-0095' 'ProstateX-0096'
 'ProstateX-0098' 'ProstateX-0104' 'ProstateX-0105' 'ProstateX-0120'
 'ProstateX-0127' 'ProstateX-0130' 'ProstateX-0134' 'ProstateX-0135'
 'ProstateX-0137' 'ProstateX-0138' 'ProstateX-0144' 'ProstateX-0146'
 'ProstateX-0147' 'ProstateX-0148' 'ProstateX-0149' 'ProstateX-0151'
 'ProstateX-0154' 'ProstateX-0155' 'ProstateX-0157' 'ProstateX-0158'
 'ProstateX-0159' 'ProstateX-0161' 'ProstateX-0163' 'ProstateX-0171'
 'ProstateX-0172' 'ProstateX-0173' 'ProstateX-0179' 'ProstateX-0180'
 'ProstateX-0188' 'ProstateX-0189'

ProxID  fid                                                pos  \
0     ProstateX-0000    1                            25.7457 31.8707 -38.511   
1     ProstateX-0000    1                            25.7457 31.8707 -38.511   
2     ProstateX-0000    1                            25.7457 31.8707 -38.511   
3     ProstateX-0000    1                            25.7457 31.8707 -38.511   
4     ProstateX-0000    1                            25.7457 31.8707 -38.511   
5     ProstateX-0000    1                            25.7457 31.8707 -38.511   
6     ProstateX-0000    1                            25.7457 31.8707 -38.511   
7     ProstateX-0000    1                            25.7457 31.8707 -38.511   
8     ProstateX-0000    1                            25.7457 31.8707 -38.511   
44    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
45    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
46    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
47    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
48    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
49    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
50    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
51    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
52    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
53    ProstateX-0003    1                           22.1495 31.2717 -2.45933   
54    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
55    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
56    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
57    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
58    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
59    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
60    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
61    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
62    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
63    ProstateX-0003    2                           -21.2871 19.3995 19.7429   
64    ProstateX-0004    1                           -7.69665 3.64226 23.1659   
...              ...  ...                                                ...   
3839  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3840  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3841  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3842  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3843  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3844  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3845  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3846  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3847  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3848  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3849  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3850  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3851  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3852  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3853  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3854  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3855  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3856  ProstateX-0203    2  10.0280456542969 -4.62264251708984 -90.3759994...   
3857  Prostate

In [189]:
training_table, testing_table = get_training_testing_tables()

We have 36 cancer samples and 159 non-cancer samples


In [34]:
training_table

ProxID  fid                                                pos  \
0   ProstateX-0120    2                              3.35409 45.6561 58.27   
1   ProstateX-0067    2                            27.2983 36.0017 16.8409   
2   ProstateX-0013    1                           -42.7915 30.7382 5.60159   
3   ProstateX-0173    2                          -18.7971 46.6287 -42.0199   
4   ProstateX-0142    2                            52.9462 53.9049 39.3076   
5   ProstateX-0025    4                           21.9659 22.9866 -63.5253   
6   ProstateX-0142    2                            52.9462 53.9049 39.3076   
7   ProstateX-0103    1                           12.9559 46.3051 -54.5266   
8   ProstateX-0123    1                            20.8643 30.9437 29.9455   
9   ProstateX-0019    1                          -4.21012 42.1916 -40.6457   
10  ProstateX-0044    1                          -5.795073 18.7907 87.5082   
11  ProstateX-0100    3                           -8.01724 38.6422 -6.1046   
12  ProstateX-0196    2                           35.2755 27.3587 -8.26241   
13  ProstateX-0037    2                           11.742 38.1655 0.0225723   
14  ProstateX-0144    3                          0.560626 31.9222 -58.6788   
15  ProstateX-0117    1                           26.7378 24.0652 -34.8602   
16  ProstateX-0099    2                           -16.1111 39.0076 65.3807   
17  ProstateX-0157    1                            -7.5415 30.6653 39.5173   
18  ProstateX-0006    2                          -9.85971 37.2739 0.561445   
19  ProstateX-0141    2                          -16.9907 23.4567 -25.1546   
20  ProstateX-0130    1                          -7.54632 40.5456 -56.3508   
21  ProstateX-0189    3                           8.35175 26.1661 -23.7111   
22  ProstateX-0124    1                           -2.33687 50.5183 61.0913   
23  ProstateX-0151    1                          -19.8161 35.8201 -65.2786   
24  ProstateX-0184    2                           25.2714 23.2703 -81.1427   
25  ProstateX-0159    1                             1.73495 46.48 -4.57566   
26  ProstateX-0130    1                          -7.54632 40.5456 -56.3508   
27  ProstateX-0159    1                             1.73495 46.48 -4.57566   
28  ProstateX-0186    2                             11.8551 32.938 20.8921   
29  ProstateX-0137    2                            12.6136 15.611 -8.18108   
..             ...  ...                                                ...   
67  ProstateX-0130    1                          -7.54632 40.5456 -56.3508   
68  ProstateX-0159    1                           -7.30205 50.7125 3.72577   
69  ProstateX-0074    1                           2.58529 47.7414 -79.8186   
70  ProstateX-0140    2                           18.8122 40.9708 -20.4472   
71  ProstateX-0196    2                           35.2755 27.3587 -8.26241   
72  ProstateX-0030    1                           35.3287 44.3674 -50.6832   
73  ProstateX-0200    2   -19.0211715698242 -35.6678161621094 -60.59195...   
74  ProstateX-0140    2                           18.8122 40.9708 -20.4472   
75  ProstateX-0112    1                           27.1693 28.6715 -8.77407   
76  ProstateX-0030    1                           35.3287 44.3674 -50.6832   
77  ProstateX-0129    1                                     42.3 39.7 20.7   
78  ProstateX-0068    2                           -22.0582 39.112 -11.8622   
79  ProstateX-0123    1                            20.8643 30.9437 29.9455   
80  ProstateX-0025    4                           21.9659 22.9866 -63.5253   
81  ProstateX-0139    3                           -5.79388 41.2189 18.3538   
82  ProstateX-0163    3                            11.7962 55.1382 19.8463   
83  ProstateX-0051    1                            -7.8659 55.0028 38.3943   
84  ProstateX-0184    1                           8.75216 26.8913 -82.9706   
85  ProstateX-0025    1                             23.6983 51.83 -55.7923   
86  ProstateX-0142    2           

In [35]:
testing_table

ProxID  fid                                                pos  \
0   ProstateX-0047    1                          -24.2868 62.7355 -24.2213   
1   ProstateX-0057    2                           26.2799 54.2679 -21.5532   
2   ProstateX-0020    1                            10.1617 53.5994 10.0223   
3   ProstateX-0029    1                            -27.064 50.9487 19.5641   
4   ProstateX-0040    2                             32.0217 37.5945 -70.54   
5   ProstateX-0093    2                           29.4178 43.3347 -62.3276   
6   ProstateX-0048    1                           18.4772 48.0755 -83.6069   
7   ProstateX-0025    2                            7.79027 54.6192 -55.072   
8   ProstateX-0116    1                           16.1963 -4.17941 25.4894   
9   ProstateX-0034    1                             17.0042 60.678 5.75149   
10  ProstateX-0108    1                            -27.5928 29.585 21.3175   
11  ProstateX-0059    1                           -26.9082 24.1523 14.1477   
12  ProstateX-0113    1                          -9.65632 52.8802 -83.0514   
13  ProstateX-0021    4                           38.7099 28.8409 -28.6634   
14  ProstateX-0093    3                            9.6081 31.5817 -68.8806   
15  ProstateX-0050    1                            11.8784 16.0017 -26.537   
16  ProstateX-0076    1                            7.23638 34.0724 71.1374   
17  ProstateX-0008    1                            25.2302 38.6613 -26.864   
18  ProstateX-0005    0  -14.5174331665039 49.4428329467773 20.78152465...   
19  ProstateX-0040    3                           41.7562 24.6357 -74.4334   
20  ProstateX-0046    1                           -32.861 -2.20032 46.9729   
21  ProstateX-0038    3                            -34.7633 44.767 14.3969   
22  ProstateX-0067    1                            18.6239 25.4505 14.9875   
23  ProstateX-0017    1                           2.89097 49.4408 -14.4697   
24  ProstateX-0081    1                          -25.8224 39.8219 -20.0141   
25  ProstateX-0025    1                             23.6983 51.83 -55.7923   
26  ProstateX-0028    2                           -9.64486 40.691 -5.07664   
27  ProstateX-0040    1                             5.7972 19.5845 -58.366   
28  ProstateX-0110    2                            11.624 16.5662 -11.7145   
29  ProstateX-0025    2                            7.79027 54.6192 -55.072   
..             ...  ...                                                ...   
31  ProstateX-0054    2                           20.3595 31.1926 -6.71699   
32  ProstateX-0057    1                              -5.013 52.868 7.27444   
33  ProstateX-0073    1                            27.3235 35.823 -40.3352   
34  ProstateX-0077    1                           -26.1229 39.9618 13.7895   
35  ProstateX-0031    2                           -25.5225 39.8319 13.5071   
36  ProstateX-0031    1                            8.08436 38.9556 13.8648   
37  ProstateX-0067    3                           -11.5378 44.1525 22.7649   
38  ProstateX-0027    1                           -8.16043 63.437 -9.60785   
39  ProstateX-0104    2                           2.70612 32.2689 -1.52071   
40  ProstateX-0103    2                          -6.37128 49.1888 -63.0611   
41  ProstateX-0022    1                          -22.5143 51.5685 -107.645   
42  ProstateX-0032    1                            19.0001 60.9885 -32.157   
43  ProstateX-0006    1                           8.20024 28.2665 -10.4784   
44  ProstateX-0106    1                            11.6356 42.7856 21.9458   
46  ProstateX-0033    1                          -0.473669 43.2903 35.9394   
47  ProstateX-0005    1                           -38.6276 42.2781 21.4084   
48  ProstateX-0065    1                           2.03432 30.3762 -21.4596   
49  ProstateX-0039    1                            -29.2205 47.977 66.1721   
50  ProstateX-0041    1                         -11.24582 60.9975 -48.0648   
51  ProstateX-0095    1           

In [196]:
rows,cols = training_table.Kimg.iloc[0].shape
print (rows,cols)
training_data = np.ndarray(shape=(len(training_table), rows, cols),
                         dtype=np.float32)
training_labels = dataset = np.ndarray(shape=(len(training_table),1),
                         dtype=np.float32)
for index in range (len(training_table)):
    training_data[index, : , : ] = training_table.Kimg.iloc[index]
    training_labels[index] = training_table.ClinSig[index]
training_data  = training_data.reshape((len(training_table),rows*cols))
training_labels = training_labels.reshape((len(training_table),))
print(training_data.shape)
print(training_labels.shape)

128 128
(97, 16384)
(97,)


In [197]:
rows,cols = testing_table.Kimg.iloc[0].shape
print (rows,cols)
testing_data = np.ndarray(shape=(len(testing_table), rows, cols),
                         dtype=np.float32)
testing_labels = dataset = np.ndarray(shape=(len(testing_table),1),
                         dtype=np.float32)
for index in range (len(testing_table)):
    testing_data[index, : , : ] = testing_table.Kimg.iloc[index]
    testing_labels[index] = testing_table.ClinSig.iloc[index]
testing_data  = testing_data.reshape((len(testing_table),rows*cols))
testing_labels = testing_labels.reshape((len(testing_table),))
print(training_data.shape)
print(training_labels.shape)

128 128
(97, 16384)
(97,)


In [207]:
classifier1 = GaussianNB()
classifier1.fit(training_data, training_labels)
result = classifier1.score(testing_data,testing_labels)
print ("This was an experiment using GaussianNB classifier and only kimages of the pheripetal zone.") 
print("the number of samples in training were {} ".format(
    len(training_table)) + "and the number of samples in testing were {} ".format(len(testing_data)))

print("the score of the accuracy in this experiment was {}".format(result))

This was an experiment using GaussianNB classifier and only kimages of the pheripetal zone.
the number of samples in training were 97 and the number of samples in testing were 60 
the score of the accuracy in this experiment was 0.7166666666666667
